In [2]:
%pip install aps-toolkit --upgrade

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\python311\lib\site-packages\autodesk_forge_sdk-0.1.2-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


### Import Libraries

In [3]:
from aps_toolkit import Auth
import requests
import os
import sqlite3

### Get Token

In [4]:
auth = Auth()

In [5]:
token = auth.auth2leg()

In [6]:
URN = "dXJuOmFkc2sud2lwcHJvZDpmcy5maWxlOnZmLnRfTFFVUDZoU3l5UXo5bm1sTE1FUHc_dmVyc2lvbj02"

In [7]:
from aps_toolkit import DbReader
db = DbReader(URN, token)


In [8]:
db.db_path

'C:\\Python311\\Lib\\site-packages\\aps_toolkit\\database\\dXJuOmFkc2sud2lwcHJvZDpmcy5maWxlOnZmLnRfTFFVUDZoU3l5UXo5bm1sTE1FUHc_dmVyc2lvbj02.sdb'

### Object ID Reader

In [9]:
db.execute_query("SELECT * FROM _objects_id")

,id,external_id,viewable_id
0,1,doc_d14ee89f-34c9-4d58-ae27-c584f63353df,None
1,2,53c5330a-0a78-42d6-8dc2-cd7dde1d46b9-001522f7,None
2,3,53c5330a-0a78-42d6-8dc2-cd7dde1d46b9-001522f8,None
3,4,53c5330a-0a78-42d6-8dc2-cd7dde1d46b9-001522f9,None
4,5,53c5330a-0a78-42d6-8dc2-cd7dde1d46b9-001522fa,None
...,...,...,...
62668,62669,57ddb049-4112-4cf4-9243-c45c24903488,None
62669,62670,0bb0c17e-ba23-4d21-8b61-4f954bea6b74,None
62670,62671,77a4e6f2-95db-4050-9ff8-c2f5ee7b22a0,None
62671,62672,0c465578-969b-4809-ae2b-d28a73a43faf,None


### Object Attribute Reader

In [10]:
db.execute_query("SELECT * FROM _objects_attr")

,id,name,category,data_type,data_type_context,description,display_name,flags,display_precision,forge_parameter
0,1,name,__name__,20,None,None,None,0,0,None
1,2,child,__child__,11,None,None,None,1,0,None
2,3,parent,__parent__,11,None,None,None,1,0,None
3,4,instanceof_objid,__instanceof__,11,None,None,None,1,0,None
4,5,hastable,__hastable__,11,None,None,None,1,0,None
...,...,...,...,...,...,...,...,...,...,...
1333,1334,W1,Other,3,autodesk.unit.unit:millimeters-1.0.1,None,W1,8,1,None
1334,1335,Frame,Materials and Finishes,20,,None,Frame,0,0,None
1335,1336,Arrow Visibility,Other,1,,None,Arrow Visibility,0,0,None
1336,1337,Quantity,Text,20,,None,Quantity,0,0,None


### Object Value Reader

In [13]:
import pandas as pd
df = db.execute_query("SELECT * FROM _objects_val")
df.head(10)

,id,value
0,1,Model
1,2,rvt
2,3,1.0
3,4,1
4,5,Revit Document
5,6,Exyte Group
6,7,"Design, Engineering & Construction"
7,8,
8,9,Issue Date
9,10,Project Status


### Object EAV Reader

In [14]:
df = db.execute_query("SELECT * FROM _objects_eav")
df.head(10)

,id,entity_id,attribute_id,value_id
0,1,1,1,1
1,2,1,9,2
2,3,1,10,3
3,4,1,8,4
4,5,1,13,5
5,6,1,14,6
6,7,1,15,7
7,8,1,16,8
8,9,1,17,8
9,10,1,18,8


### Read SQLite file

In [11]:
# read sqlite file over all tables
db_file = db.db_path
try:
    # Connect to the SQLite database
    conn = sqlite3.connect(db_file)

    # Create a cursor object to execute SQL queries
    cursor = conn.cursor()
    # Query the sqlite_master table to get the list of all tables
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

    # Fetch the results
    tables = cursor.fetchall()

    # Extract table names and print them
    for table in tables:
        print(table[0])

except sqlite3.Error as e:
    print(f"SQLite error: {e}")
finally:
    # Close the database connection when done
    conn.close()

_objects_id
_objects_attr
_objects_val
_objects_eav


In [16]:

# Define and execute the SQL query with JOIN statements
query = """
SELECT _objects_id.external_id AS externalId, _objects_attr.category AS category,_objects_attr.display_name, _objects_val.value
FROM _objects_id
JOIN _objects_eav ON _objects_id.id = _objects_eav.entity_id
JOIN _objects_attr ON _objects_eav.attribute_id = _objects_attr.id
JOIN _objects_val ON _objects_eav.value_id = _objects_val.id
WHERE category NOT LIKE '\_\_%\_\_' ESCAPE '\\' 
ORDER BY externalId;
"""
data = db.execute_query(query)
data.head(20)

,externalId,category,display_name,value
0,00c6c04a-12df-4dad-941f-b193ceb216ca-0000225e,Identity Data,Image,
1,00c6c04a-12df-4dad-941f-b193ceb216ca-0000225e,Identity Data,Keynote,
2,00c6c04a-12df-4dad-941f-b193ceb216ca-0000225e,IFC Parameters,Export to IFC,By Type
3,00c6c04a-12df-4dad-941f-b193ceb216ca-0000225e,IFC Parameters,Export to IFC As,
4,00c6c04a-12df-4dad-941f-b193ceb216ca-0000225e,IFC Parameters,IFC Predefined Type,
5,00c6c04a-12df-4dad-941f-b193ceb216ca-0000225e,IFC Parameters,IfcGUID,00ni1A4jzDhPGViPFEiZIK
6,00c6c04a-12df-4dad-941f-b193ceb216ca-0000225e,Identity Data,Model,
7,00c6c04a-12df-4dad-941f-b193ceb216ca-0000225e,Identity Data,Manufacturer,
8,00c6c04a-12df-4dad-941f-b193ceb216ca-0000225e,Identity Data,Comments,
9,00c6c04a-12df-4dad-941f-b193ceb216ca-0000225e,Identity Data,URL,


### Get All Categories

In [30]:
# Define and execute the SQL query with JOIN statements
query = """
SELECT _objects_id.external_id, _objects_attr.category,_objects_attr.display_name, _objects_val.value
FROM _objects_id
JOIN _objects_eav ON _objects_id.id = _objects_eav.entity_id
JOIN _objects_attr ON _objects_eav.attribute_id = _objects_attr.id
JOIN _objects_val ON _objects_eav.value_id = _objects_val.id
WHERE _objects_attr.name LIKE '_RC';
"""
df_categories = pd.read_sql_query(query, conn)
# get unique values from the category column
categories = df_categories['value'].unique()
# create a dataframe for categories
df_categories = pd.DataFrame(categories, columns=['category'])
df_categories.head(50)
df_categories.sort_values(by=['category'], inplace=True)
df_categories.head(50)

,category
0,
35,<Sketch>
53,Air Terminals
5,Area Schemes
43,Automatic Sketch Dimensions
19,Building Type Settings
14,Cameras
61,Center Line
54,Center line
17,Color Fill Schema


In [17]:
conn = sqlite3.connect(db_file)
# Define and execute the SQL query with JOIN statements
query = """
SELECT _objects_id.external_id, _objects_attr.category,_objects_attr.display_name, _objects_val.value
FROM _objects_id
JOIN _objects_eav ON _objects_id.id = _objects_eav.entity_id
JOIN _objects_attr ON _objects_eav.attribute_id = _objects_attr.id
JOIN _objects_val ON _objects_eav.value_id = _objects_val.id
WHERE _objects_attr.name LIKE '_RC';
"""
df_categories = db.execute_query(query)
# get unique values from the category column
categories = df_categories['value'].unique()
categories

array(['', 'Materials', 'Elevations', 'Views', 'Primary Contours',
       'Area Schemes', 'Phases', 'Shared Site', 'Work Plane Grid',
       'Revision', 'Project Information', 'Structural Load Cases',
       'Sun Path', 'Internal Origin', 'Cameras', 'Section Boxes',
       'HVAC Zones', 'Color Fill Schema', 'HVAC Load Schedules',
       'Building Type Settings', 'Space Type Settings', 'Survey Point',
       'Project Base Point', 'Electrical Demand Factor Definitions',
       'Electrical Load Classifications',
       'Panel Schedule Templates - Branch Panel',
       'Panel Schedule Templates - Data Panel',
       'Panel Schedule Templates - Switchboard',
       'Electrical Load Classification Parameter Element',
       'Material Assets', 'Pipe Segments', 'Legend Components',
       'Generic Annotations', 'Lines', 'Detail Items', '<Sketch>',
       'Rooms', 'Spaces', 'Title Blocks', 'Schedules', 'Text Notes',
       'Dimensions', 'Constraints', 'Automatic Sketch Dimensions',
       'Revi